In [ ]:
import tensorflow as tf

import keras
import keras.layers as kl
from keras.models import Model, load_model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, History, ModelCheckpoint

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn

In [ ]:
data = pd.read_csv('data/A549.data.csv.gz')

data

In [ ]:
# define a dictionary to map nucleotides to their one-hot encoded representation
nucleotide_dict = {'A': [1, 0, 0, 0],
                   'C': [0, 1, 0, 0],
                   'G': [0, 0, 1, 0],
                   'T': [0, 0, 0, 1],
                   'N': [0, 0, 0, 0]}

# define a function to one-hot encode a single DNA sequence
def one_hot_encode_seq(seq):
    return np.array([nucleotide_dict[nuc] for nuc in seq])

# function to load sequences and enhancer activity
def prepare_input(data_set):
    # one-hot encode DNA sequences, apply function
    seq_matrix = np.array(data_set['seq'].apply(one_hot_encode_seq).tolist())
    print(seq_matrix.shape) # dimensions are (number of sequences, length of sequences, nucleotides)
    
    # Get output array
    Y = data_set['category'].map({'enhancer' : 1, 'negative' : 0, 'neutral' : 0})
    
    return seq_matrix, Y

In [ ]:
X_train, Y_train = prepare_input(data[data['set'] == "Train"])
X_valid, Y_valid = prepare_input(data[data['set'] == "Valid"])
X_test, Y_test = prepare_input(data[data['set'] == "Test"])

In [ ]:
params = {'batch_size': 128,
          'epochs': 100,
          'early_stop': 20,
          'lr': 0.001,
          'n_conv_layer': 3,
          'num_filters1': 64,
          'num_filters2': 16,
          'num_filters3': 16,
          'kernel_size1': 7,
          'kernel_size2': 5,
          'kernel_size3': 5,
          'n_dense_layer': 2,
          'dense_neurons1': 32,
          'dense_neurons2': 64,
          'dropout_conv': 'no',
          'dropout_prob': 0.4,
          'pad':'same'}


def CNNet(params):

    # expects sequences of length 350 with 4 channels, length of DNA sequences
    input = kl.Input(shape=(350, 4))

    # Body - 4 conv + batch normalization + ReLu activation + max pooling
    x = kl.Conv1D(params['num_filters1'], kernel_size=params['kernel_size1'],
                  padding=params['pad'],
                  name='Conv1D_1')(input)
    x = kl.BatchNormalization()(x)
    x = kl.Activation('relu')(x)
    x = kl.MaxPooling1D(2)(x)

    for i in range(1, params['n_conv_layer']):
        x = kl.Conv1D(params['num_filters'+str(i+1)],
                      kernel_size=params['kernel_size'+str(i+1)],
                      padding=params['pad'],
                      name=str('Conv1D_'+str(i+1)))(x)
        x = kl.BatchNormalization()(x)
        x = kl.Activation('relu')(x)
        x = kl.MaxPooling1D(2)(x)
        # add dropout after convolutional layers?
        if params['dropout_conv'] == 'yes': x = kl.Dropout(params['dropout_prob'])(x)

    # After the convolutional layers, the output is flattened and passed through a series of fully connected/dense layers
    # Flattening converts a multi-dimensional input (from the convolutions) into a one-dimensional array (to be connected with the fully connected layers
    x = kl.Flatten()(x)

    # Fully connected layers
    # Each fully connected layer is followed by batch normalization, ReLU activation, and dropout
    for i in range(0, params['n_dense_layer']):
        x = kl.Dense(params['dense_neurons'+str(i+1)],
                     name=str('Dense_'+str(i+1)))(x)
        x = kl.BatchNormalization()(x)
        x = kl.Activation('relu')(x)
        x = kl.Dropout(params['dropout_prob'])(x)

    # Model output
    output = kl.Dense(1, activation='sigmoid', name=str('Dense_output'))(x)

    return Model(input, output)

In [ ]:
# Create model with specified parameters
model = CNNet(params)

model.summary()

model.compile(optimizer=Adam(learning_rate=params['lr']),
              loss='binary_crossentropy',
              metrics=['accuracy'])

train_history = model.fit(X_train, Y_train,
                          validation_data=(X_valid, Y_valid),
                          batch_size=params['batch_size'],
                          epochs=params['epochs'],
                          callbacks=[EarlyStopping(patience=params['early_stop'], monitor="val_loss", restore_best_weights=True), History()])

In [ ]:
# Plot training & validation training metrics

plt.plot(train_history.history[str('loss')])
plt.plot(train_history.history[str('val_loss')])
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Add vertical line at minimum validation loss of combined dev and hk
min_val_loss = min(train_history.history['val_loss'])
plt.axvline(x=train_history.history['val_loss'].index(min_val_loss), color='red', linestyle='--')

plt.show()

In [ ]:
score = model.evaluate(X_train, Y_train, verbose=0)
print("Train loss:", score[0])
print("Train accuracy:", score[1])

score = model.evaluate(X_valid, Y_valid, verbose=0)
print("Valid loss:", score[0])
print("Valid accuracy:", score[1])

score = model.evaluate(X_test, Y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
model.save('model.h5')

In [ ]:
import shap
from deeplift.visualization import viz_sequence

# select a set of background examples to take an expectation over
np.random.seed(seed=123)
background = X_train[np.random.choice(X_train.shape[0], 1000, replace=False)]

# Prepare DeepExplainer for model output
shap.explainers._deep.deep_tf.op_handlers["AddV2"] = shap.explainers._deep.deep_tf.passthrough # this is required due to conflict between versions (https://github.com/slundberg/shap/issues/1110)
explainer = shap.DeepExplainer(model, background)

In [ ]:
### Compute nucleotide contribution scores

# get strong enhancers
cand_enh = data.loc[(data['set'] == 'Test') &
                    (data['pvalue'] < 0.05) &
                    (data['log2FC'] > 1)]

# sort by strength to have the stongest on top for later plotting
cand_enh = cand_enh.sort_values('log2FC', ascending = False)

# one-hot encode sequences
X_cand_enh, Y_cand_enh = prepare_input(cand_enh)

# Predict with model
pred_values_enh = model.predict(X_cand_enh).squeeze()

# calculate scores, get first element of output
shap_values_enh = explainer.shap_values(X_cand_enh)[0]

final_contr_scores = shap_values_enh * X_cand_enh

In [ ]:
for i in range(5):
    print('Enhancer:', cand_enh.iloc[i]['name'],
          ' / Obs act:', '{0:0.2f}'.format(cand_enh.iloc[i]['log2FC']),
          ' / Pred prob:','{0:0.2f}'.format(pred_values_enh[i]))
    
    print('Actual contribution scores')
    viz_sequence.plot_weights(final_contr_scores[i], figsize=(20, 2), subticks_frequency=20)

    # print('Hypothetical contribution scores')
    # viz_sequence.plot_weights(shap_values_enh[i], figsize=(20, 2), subticks_frequency=20)

In [ ]:
import modisco
import modisco.affinitymat.core
import modisco.cluster.phenograph.core
import modisco.cluster.phenograph.cluster
import modisco.cluster.core
import modisco.aggregator
import modisco.util
from modisco.visualization import viz_sequence

from collections import OrderedDict
from collections import Counter

import numpy as np
import h5py
import inspect


# check parameters
def get_default_args(func):
    signature = inspect.signature(func)
    return {
        k: v.default
        for k, v in signature.parameters.items()
        if v.default is not inspect.Parameter.empty
    }

get_default_args(modisco.tfmodisco_workflow.workflow.TfModiscoWorkflow)

In [ ]:
task_to_scores = OrderedDict()
task_to_scores['enhancers'] = final_contr_scores
task_to_hyp_scores = OrderedDict()
task_to_hyp_scores['enhancers'] = shap_values_enh

null_per_pos_scores = modisco.coordproducers.LaplaceNullDist(num_to_samp=5000)


# prepare TF-modisco function to run
def tfmodisco(X):

    one_hot = X

    tfmodisco_results = modisco.tfmodisco_workflow.workflow.TfModiscoWorkflow(
                        #target_seqlet_fdr controls the stringency of the threshold used.
                        # the default value is 0.2
                        target_seqlet_fdr=0.2,
                        #min_passing_windows_frac and max_passing_windows_frac can be used
                        # to manually adjust the percentile cutoffs for importance
                        # scores if you feel that the cutoff
                        # defined by the null distribution is too stringent or too
                        # lenient. The default values are 0.03 and 0.2 respectively.
                        #min_passing_windows_frac=0.03,
                        #max_passing_windows_frac=0.2
                        #The sliding window size and flanks should be adjusted according to the expected length of the core motif and its flanks.
                        #If the window size or flank sizes are too long, you risk picking up more noise.
                        sliding_window_size=15,
                        flank_size=5,
                        max_seqlets_per_metacluster=50000,
                        seqlets_to_patterns_factory=
                            modisco.tfmodisco_workflow
                                    .seqlets_to_patterns
                                    .TfModiscoSeqletsToPatternsFactory(
                                #kmer_len, num_gaps and num_mismatches are used to
                                # derive kmer embeddings for coarse-grained affinity
                                # matrix calculation. kmer_len=6, num_gaps=1
                                # and num_mismatches=0 means
                                # that kmer embeddings using 6-mers with 1 gap will be
                                # used. The default is to use longer kmers, but this
                                # can take a while to run and can lead to
                                # out-of-memory errors on some systems.
                                # Empirically, 6-mers with 1-gap
                                # seem to give good results.
                                #During the seqlet clustering, motifs are trimmed to the central trim_to_window_size bp with the highest importance
                                trim_to_window_size=15,
                                #After the trimming is done, the seqlet is expanded on either side by initial_flank_to_add
                                initial_flank_to_add=5,
                                final_min_cluster_size=50
                        )
                   )(
                task_names=['enhancers'],
                contrib_scores=task_to_scores,
                hypothetical_contribs=task_to_hyp_scores,
                one_hot=one_hot,
                null_per_pos_scores = null_per_pos_scores)

    return tfmodisco_results

In [ ]:
# function to visualize motifs

def modisco_motif_plots(hdf5_modisco_results):

    hdf5_results = h5py.File(hdf5_modisco_results, "r")

    metacluster_names = [
        x.decode("utf-8") for x in
        list(hdf5_results["metaclustering_results"]["all_metacluster_names"][:])]

    all_patterns = []

    # sequence background
    background = np.mean(X_cand_enh, axis=(0, 1))


    for metacluster_name in metacluster_names:
        print(metacluster_name)
        metacluster_grp = (hdf5_results["metacluster_idx_to_submetacluster_results"]
                                       [metacluster_name])
        print("activity pattern:",metacluster_grp["activity_pattern"][:])
        all_pattern_names = [x.decode("utf-8") for x in
                             list(metacluster_grp["seqlets_to_patterns_result"]
                                                 ["patterns"]["all_pattern_names"][:])]
        if (len(all_pattern_names)==0):
            print("No motifs found for this activity pattern")
        for pattern_name in all_pattern_names:
            print(metacluster_name, pattern_name)
            all_patterns.append((metacluster_name, pattern_name))
            pattern = metacluster_grp["seqlets_to_patterns_result"]["patterns"][pattern_name]
            print("total seqlets:",len(pattern["seqlets_and_alnmts"]["seqlets"]))
            # print("Actual importance scores:")
            # viz_sequence.plot_weights(pattern[str(task + "_contrib_scores")]["fwd"])
            # print("Hypothetical scores:")
            # viz_sequence.plot_weights(pattern[str(task + "_hypothetical_contribs")]["fwd"])
            print("IC-scaled, fwd and rev:")
            viz_sequence.plot_weights(viz_sequence.ic_scale(np.array(pattern["sequence"]["fwd"]),
                                                            background=background))
            viz_sequence.plot_weights(viz_sequence.ic_scale(np.array(pattern["sequence"]["rev"]),
                                                            background=background))

            #Plot the subclustering too, if available
            if ("subclusters" in pattern):
                print("PLOTTING SUBCLUSTERS")
                subclusters = np.array(pattern["subclusters"])
                twod_embedding = np.array(pattern["twod_embedding"])
                plt.scatter(twod_embedding[:,0], twod_embedding[:,1], c=subclusters, cmap="tab20")
                plt.show()
                for subcluster_name in list(pattern["subcluster_to_subpattern"]["subcluster_names"]):
                    subpattern = pattern["subcluster_to_subpattern"][subcluster_name]
                    print(subcluster_name.decode("utf-8"), "size", len(subpattern["seqlets_and_alnmts"]["seqlets"]))
                    subcluster = int(subcluster_name.decode("utf-8").split("_")[1])
                    plt.scatter(twod_embedding[:,0], twod_embedding[:,1], c=(subclusters==subcluster))
                    plt.show()
                    # viz_sequence.plot_weights(subpattern[str(task + "_hypothetical_contribs")]["fwd"])
                    # viz_sequence.plot_weights(subpattern[str(task + "_contrib_scores")]["fwd"])
                    viz_sequence.plot_weights(viz_sequence.ic_scale(np.array(subpattern["sequence"]["fwd"]),
                                                            background=background))

    hdf5_results.close()

In [ ]:
# Run TF-Modisco
tfmodisco_results = tfmodisco(X_cand_enh)

# save results
hdf5_modisco_results = "modisco_results.hdf5"

grp = h5py.File(hdf5_modisco_results, "w")
tfmodisco_results.save_hdf5(grp)
grp.close()

In [ ]:
# Visualize motifs
modisco_motif_plots(hdf5_modisco_results)